In [111]:
import pandas as pd
import numpy as np
import math
from io import StringIO

#NOTES
#
#Get rid of info after comma in names
#

In [2]:
def squeeze_nan_left(x):
            original_columns = x.index.tolist()
            squeezed = x.dropna()
            squeezed.index = [original_columns[n] for n in range(squeezed.count())]
            return squeezed.reindex(original_columns, fill_value=np.nan)

In [3]:
def SCT_table_format(SCT):

            return SCT

In [132]:
EVRI = 'https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm'
MNTA = 'https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm'
IIVI = 'https://www.sec.gov/Archives/edgar/data/820318/000119312517286341/d434985ddef14a.htm'
OPHT = 'https://www.sec.gov/Archives/edgar/data/1410939/000141093918000020/proxystatementfor2018annua.htm'

url = MNTA
ticker = 'IIVI'


filing = pd.read_html(url)
#filing[:]
#168 for Everi
#128 for MNTA
#20 for OPHT
#X IIVI

filing[128]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Name and Principal Position,Year,Salary(1)($),Stock Awards(2) ($),Option Awards(2) ($),Non‑Equity Incentive Plan Compensation ($),All Other Compensation(3) ($),Total ($),NaN,NaN,NaN,NaN,NaN,NaN
3,Craig A. Wheeler,2016,689179,NaN,2149290,NaN,737818,NaN,554789,NaN,29948.0,NaN,4161024.0,NaN
4,"President, Chief Executive Officer and Director",2015,691484,NaN,3064900,NaN,1046052,NaN,512723,NaN,29503.0,NaN,5344662.0,NaN
5,2014,643356,NaN,1257200,NaN,1558564,NaN,432335,NaN,29610.0,NaN,3921065.0,NaN,NaN
6,Scott M. Storer (4),2016,40385,NaN,684000,NaN,940375,NaN,—,NaN,150.0,NaN,1664910.0,NaN
7,Senior Vice President and Chief Financial Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Matthew P. Ottmer (5)(6),2016,450000,NaN,733500,NaN,—,NaN,252281,NaN,13091.0,NaN,1448872.0,NaN


In [134]:
filing[128].iloc[:,3].str.contains('Name').tolist().count(True)


0

In [135]:
filing_sct_score = [0]*len(filing)

for i in range(0,len(filing_sct_score)):
    for j in range(0,filing[i].shape[0]):
        if filing[128].iloc[:,j].str.contains('Name|Position|Salary|Stock|Award|Option|Non‑Equity|Incentive|Plan|Compensation|All Other|Total|').tolist().count(True) > 0:
            filing_sct_score[i] = filing_sct_score[i] + 1
        
    
filing_sct_score
#sctind = 128
#SCTo = filing[sctind]
#SCTo

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [42]:
#Table Formatting tests

#initial format, take out nan rows/cols and squeeze nan left
SCT = SCTo.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

#secondary format - fill dashes with 0s, remove symbols and parenthesis objects
for i in range(0,SCT.shape[0]):
    for j in range(0,SCT.shape[1]):
        if type(SCT.iloc[i,j]) == str:
            if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                SCT.iloc[i,j] = 0
            elif '(' in SCT.iloc[i,j]:
                tempstr = SCT.iloc[i,j].split("(")
                tempstr2 = tempstr[0]
                SCT.iloc[i,j] = tempstr2
            elif len(SCT.iloc[i,j]) < 2:
                SCT.iloc[i,j] = np.nan             

SCT = SCT.reset_index(drop=True)


#correct for years that are bumped too far left
for i in range(0,SCT.shape[0]):
    if pd.isnull(SCT.iloc[i, SCT.shape[1]-1]):
        rightslip = SCT.iloc[i].tolist()
        rightslip.insert(0,'HOLD')
        SCT.iloc[i] = rightslip[:-1]
        

#correct for positions moved over to the right
for i in range(0,SCT.shape[0]):
    if SCT.iloc[i].isnull().values.any():
        leftslip = SCT.iloc[i].tolist()
        leftslip.insert(SCT.shape[1]-1,np.nan)
        leftslip.remove("HOLD")
        SCT.iloc[i] = leftslip
        
SCT.replace(to_replace = 'HOLD', value = '', inplace = True)

#don't do this, screws up Name df below
#SCT = SCT.fillna(value = '')

#update column names
SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 0)                

SCT = SCT.reset_index(drop=True)
SCT


,Name and Principal Position,Year,Salary,Stock Awards,Option Awards,Non‑Equity Incentive Plan Compensation,All Other Compensation,Total
0,Craig A. Wheeler,2016,689179,2149290,737818,554789,29948,4.16102e+06
1,"President, Chief Executive Officer and Director",2015,691484,3064900,1046052,512723,29503,5.34466e+06
2,,2014,643356,1257200,1558564,432335,29610,3.92106e+06
3,Scott M. Storer,2016,40385,684000,940375,0,150,1.66491e+06
4,Senior Vice President and Chief Financial Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Matthew P. Ottmer,2016,450000,733500,0,252281,13091,1.44887e+06
6,Chief Operating Officer,2015,34615,0,1507975,0,29,1.54262e+06
7,Ganesh V. Kaundinya,2016,433350,791487,221345,201833,13091,1.66111e+06
8,Chief Scientific Officer and Senior Vice Presi...,2015,432708,927139,310065,187924,13014,1.87085e+06
9,,2014,400656,346844,429440,163468,12852,1.35326e+06


In [44]:
for i in range(SCT.shape[0]):
    for j in range(SCT.shape[1]):
        SCT.iloc[i,j] = pd.to_numeric(SCT.iloc[i,j], errors = 'ignore')

In [68]:
#creating NEO name/position lists - done after table format
#consider making a version that chops off duplicate names, middle initial, endings like M.D.

NEO_Names = []

for i in range(0,SCT.shape[0]):
    if SCT.iloc[i,1] == SCT.iloc[:,1].max():
        NEO_Names.append(SCT.iloc[i,0])
        
        
NEO_Combine = pd.DataFrame(NEO_Names)
NEO_Combine.columns = ['Name']
SCT_Positions = SCT.iloc[:,0].dropna().reset_index(drop=True)
NEO_Combine['Position'] = ''
NEO_Names


['Craig A. Wheeler',
 'Scott M. Storer ',
 'Matthew P. Ottmer ',
 'Ganesh V. Kaundinya',
 'Bruce A. Leicher',
 'Richard P. Shea ']

In [69]:
k = 0

for i in range(1, len(SCT_Positions)):
    if SCT_Positions[i] in NEO_Names:
        k = k + 1
        i = i + 1
    else:
        NEO_Combine.iloc[k,1] = NEO_Combine.iloc[k,1] + ' ' + SCT_Positions[i]
        i = i + 1


NEO_Combine

,Name,Position
0,Craig A. Wheeler,"President, Chief Executive Officer and Director"
1,Scott M. Storer,Senior Vice President and Chief Financial Off...
2,Matthew P. Ottmer,Chief Operating Officer
3,Ganesh V. Kaundinya,Chief Scientific Officer and Senior Vice Pres...
4,Bruce A. Leicher,"Senior Vice President, General Counsel and Se..."
5,Richard P. Shea,Senior Vice President and Chief Financial Off...


In [72]:
#Putting it all together

#throw the warning for copying
pd.options.mode.chained_assignment = None  # default='warn'


SCT_Final = SCT.iloc[:,1:].dropna().reset_index(drop=True)
SCT_Final.insert(loc = 0, column = 'Ticker', value = ticker)
SCT_Final.insert(loc = 1, column = 'Name', value = np.nan)
SCT_Final.insert(loc = 2, column = 'Position', value = np.nan)

#this could screw up the rearragement
#SCT_Final.columns['Ticker', 'Name', 'Position', 'Year', 'Salary', 'Bonus', 'Stock Awards', 'Option Awards', 'Non- Equity Incentive Plan Compensation', 'All Other Compensation', 'Total']

k = 0

for i in range(0, SCT_Final.shape[0]):
    if (i>0) and (SCT_Final['Year'][i] >= SCT_Final['Year'][i-1]):
        k = k + 1
    SCT_Final['Name'][i] = NEO_Combine['Name'][k]
    SCT_Final['Position'][i] = NEO_Combine['Position'][k]
    

SCT_Final

,Ticker,Name,Position,Year,Salary,Stock Awards,Option Awards,Non‑Equity Incentive Plan Compensation,All Other Compensation,Total
0,MNTA,Craig A. Wheeler,"President, Chief Executive Officer and Director",2016,689179,2149290,737818,554789,29948,4.16102e+06
1,MNTA,Craig A. Wheeler,"President, Chief Executive Officer and Director",2015,691484,3064900,1046052,512723,29503,5.34466e+06
2,MNTA,Craig A. Wheeler,"President, Chief Executive Officer and Director",2014,643356,1257200,1558564,432335,29610,3.92106e+06
3,MNTA,Scott M. Storer,Senior Vice President and Chief Financial Off...,2016,40385,684000,940375,0,150,1.66491e+06
4,MNTA,Matthew P. Ottmer,Chief Operating Officer,2016,450000,733500,0,252281,13091,1.44887e+06
5,MNTA,Matthew P. Ottmer,Chief Operating Officer,2015,34615,0,1507975,0,29,1.54262e+06
6,MNTA,Ganesh V. Kaundinya,Chief Scientific Officer and Senior Vice Pres...,2016,433350,791487,221345,201833,13091,1.66111e+06
7,MNTA,Ganesh V. Kaundinya,Chief Scientific Officer and Senior Vice Pres...,2015,432708,927139,310065,187924,13014,1.87085e+06
8,MNTA,Ganesh V. Kaundinya,Chief Scientific Officer and Senior Vice Pres...,2014,400656,346844,429440,163468,12852,1.35326e+06
9,MNTA,Bruce A. Leicher,"Senior Vice President, General Counsel and Se...",2016,414655,771993,200265,193126,12998,1.59304e+06


In [73]:
#Print to excel

SCT_Final.to_excel('test.xlsx', sheet_name='Data', index=False)
